<a href="https://colab.research.google.com/github/umeshsureban/AI-For-Beginners/blob/main/Travel_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Step-by-Step Guide to Building AI Agent Teams with CrewAI


###Introduction
CrewAI is a powerful framework that allows you to create, coordinate and deploy teams of AI agents that work together to accomplish complex tasks. This guide will walk you through the process of creating your own AI agent teams, even if you have minimal coding experience.

###install Required Libraries

In [ ]:
!pip install -qU crewai langchain_openai 'crewai[tools]' google-search-results

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

###Getting Your API Keys
**Serper API Key**

* Visit https://serper.dev/api-key
* Sign up for an account or log in
* Navigate to the API dashboard
* Generate your API key
* Copy the key and use in your Colab Notebook

**Google Gemini API Key**

* Visit https://aistudio.google.com/
 Sign in with your Google account
* Navigate to "API Keys" in the settings menu
* Click "Create API Key"
* Copy your new API key And use in Colab Notebook


**How to Configure API Keys On Google Colab**
* Click on the key icon in the left sidebar
* Select "Add new Secret"
* Add your API keys with appropriate names: GEMINI_API_KEY, SERPER_API_KEY


###Setup API Keys

In [ ]:
import os
from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

###Initialize Models

In [ ]:
from langchain_openai import ChatOpenAI
from crewai import LLM

Gemini = LLM(
    model="gemini/gemini-2.0-flash",)


#gpt4o = ChatOpenAI(model = "gpt-4o")

###1. AI Travel Agent 🧳


In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

# Initialize tools
search_tool = SerperDevTool()

def create_travel_agent(destination, preferences):
    # Define the Travel Agent
    travel_agent = Agent(
        role="Travel Planner",
        goal=f"Plan a trip to {destination} based on user preferences",
        backstory="An expert travel assistant trained to create personalized travel itineraries",
        verbose=True,
        tools=[search_tool],
        llm=Gemini
    )

    # Define the Local Expert Agent
    local_expert = Agent(
        role="Local Expert",
        goal=f"Provide insider knowledge and cultural insights about {destination}",
        backstory=f"A longtime resident and cultural expert of {destination} with deep knowledge of local customs, hidden gems, and authentic experiences",
        verbose=True,
        tools=[search_tool],
        llm=Gemini
    )

    # Define the Travel Planning Task
    travel_task = Task(
        description=f"Find the best places to visit, stay, and eat in {destination} based on preferences: {preferences}",
        expected_output=f"A detailed travel itinerary for {destination} including places to visit, accommodations, and restaurants.",
        agent=travel_agent
    )

    # Define the Local Insights Task
    local_task = Task(
        description=f"Provide insider tips, cultural insights, and hidden gems for {destination}. Consider local customs, best times to visit specific locations, and authentic local experiences.",
        expected_output=f"A comprehensive guide of insider knowledge about {destination}, including cultural tips, local customs, and secret spots not found in typical tourist guides.",
        agent=local_expert
    )

    # Create and Run the Crew
    crew = Crew(
        agents=[travel_agent, local_expert],
        tasks=[travel_task, local_task],
        verbose=True,
        process=Process.sequential
    )

    result = crew.kickoff(inputs={"destination": destination, "preferences": preferences})
    return result

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

### Execute the agent


In [ ]:
destination = input("Enter your travel destination (e.g., Paris, Bali): ")
preferences = input("Enter your preferences (e.g., adventure, luxury, budget-friendly): ")
travel_result = create_travel_agent(destination, preferences)
print("Travel Itinerary:")
print(travel_result)

Enter your travel destination (e.g., Paris, Bali): paris
Enter your preferences (e.g., adventure, luxury, budget-friendly): adventure
# Agent: Travel Planner
## Task: Find the best places to visit, stay, and eat in paris based on preferences: adventure


# Agent: Travel Planner
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"adventure activities in Paris\"}"
## Tool Output: 
{'searchParameters': {'q': 'adventure activities in Paris', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'THE 10 BEST Adventurous Things to Do in Paris (2025) - Tripadvisor', 'link': 'https://www.tripadvisor.com/Attractions-g187147-Activities-zft12163-Paris_Ile_de_France.html', 'snippet': 'Recommended Adventurous Experiences (108) · 3. THATLou, Treasure Hunt at the Louvre · 4. Retro Tour Paris · 5. Comptoir des catacombes · 6. Paris by Scooter · 7 ...', 'position': 1}, {'title': '10 thrilling things to do in Paris!', 'link': 'https://parisjetaime.com/e